
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Le projet a été développé à l'aide de Alice Breton, étudiante à la maîtrise en génie informatique. Elle a suivi le cours lors de la session Hiver 2019.



# Projet : Débordement d'égouts

La description du projet est disponible à l'adresse suivante :
https://www.kaggle.com/t/a238b752c33a41d9803c2cdde6bfc929

Ce calepin Jupyter de base permet de charger et de nettoyer les données fournies. La dernière section détaille la génération du fichier des prédictions afin de le soumettre sur Kaggle dans le bon format.

Dans un premier temps, vous devrez récupérer l'archive *data.zip* sur Moodle. Ce dossier contient les fichiers suivants :
- surverses.csv
- precipitation.csv
- ouvrages-surverses.csv
- test.csv

Veuillez le décompresser dans le répertoire de ce calepin.

Le fichier *surverse.csv* répertorie s'il y a surverse (1) ou non (0) au cours de la journée pour les 170 ouvrages de débordement de 2013 à 2018 pour les mois de mai à octobre (inclusivement). Des renseignements additionnels sur les données sont disponibles à l'adresse suivante :

http://donnees.ville.montreal.qc.ca/dataset/debordement


Le fichier *precipitation.csv* contient les précipitations horaires en dixième de *mm* enregistrées à 5 stations pluviométriques de 2013 à 2019 :
- McTavish (7024745)
- Ste-Anne-de-Bellevue (702FHL8)
- Montreal/Pierre Elliott Trudeau Intl (702S006)
- Montreal/St-Hubert (7027329)
- L’Assomption (7014160)

Plus d'informations sur les précipitations sont disponibles à l'adresse suivante :

https://climat.meteo.gc.ca/climate_data/hourly_data_f.html?hlyRange=2008-01-08%7C2019-11-12&dlyRange=2002-12-23%7C2019-11-12&mlyRange=%7C&StationID=30165&Prov=QC&urlExtension=_f.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=17&searchMethod=contains&Month=11&Day=12&txtStationName=montreal&timeframe=1&Year=2019

Le fichier *ouvrages-surverses.csv* contient différentes caractéristiques des ouvrages de débordement. 

http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse

Le fichier *test.csv* contient les ouvrages et les jours pour lesquels vous devez prédire s'il y a eu surverse (true) ou non (false). Notez que l'on s'intéresse ici à 5 ouvrages de débordement localisés tout autour de l'Ile de Montréal :
- 3260-01D dans Rivière-des-Prairies 
- 3350-07D dans Ahunstic 
- 4240-01D dans Pointe-aux-Trembles 
- 4350-01D dans le Vieux-Montréal 
- 4380-01D dans Verdun

#### Remarque

Dans le projet, on ne s'intéresse qu'aux surverses occasionnées par les précipitations. On ignore les surverses occasionnées par 
- fonte de neige (F)
- travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

On suppose que lorsqu'il n'y a pas de raison pour la surverse, il s'agit d'une surverse causée par les précipitations. Puisque Nous nous intéresserons uniquement aux surverses occasionnées par les précipitations liquides, nous ne considérons que les mois de mai à octobre inclusivement.

In [4]:
using CSV, DataFrames, Statistics, Dates, Gadfly, GLM, Distributions, LinearAlgebra

# Chargement des données et nettoyage préliminaire

## Chargement des surverses

In [5]:
data_surverse_init = CSV.read("data/surverses.csv",missingstring="-99999")
first(data_surverse_init,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


## Nettoyage des données sur les surverses

#### Extraction des surverses pour les mois de mai à octobre inclusivement

In [6]:
data_surverse_init = filter(row -> month(row.DATE) > 4, data_surverse_init) 
data_surverse_init = filter(row -> month(row.DATE) < 11, data_surverse_init) 
first(data_surverse_init,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


#### Remplacement des valeurs *missing* dans la colonne :RAISON par "Inconnue"

In [7]:
raison = coalesce.(data_surverse_init[:,:RAISON],"Inconnue")
data_surverse_init[!,:RAISON] = raison
first(data_surverse_init,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String
1,0642-01D,2013-05-01,0,Inconnue
2,0642-01D,2013-05-02,0,Inconnue
3,0642-01D,2013-05-03,0,Inconnue
4,0642-01D,2013-05-04,0,Inconnue
5,0642-01D,2013-05-05,0,Inconnue


#### Exlusion des surverses coccasionnées par d'autres facteurs que les précipitations liquides

Ces facteurs correspondent à : 
- la fonte de neige (F), 
- les travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

In [8]:
data_surverse = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data_surverse_init) 
select!(data_surverse, [:NO_OUVRAGE, :DATE, :SURVERSE])
select!(data_surverse_init, [:NO_OUVRAGE, :DATE, :SURVERSE])
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64⍰
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


#### Exclusion des lignes où :SURVERSE est manquante

In [9]:
data_surverse_init = dropmissing(data_surverse_init, disallowmissing=true)
data_surverse = dropmissing(data_surverse, disallowmissing=true)
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


#### On ne garde que les données de surverse concernant lesouvrages qui nous intéressent

In [10]:
ouvrage1 = "4350-01D"
ouvrage2 = "3260-01D"
ouvrage3 = "3350-07D"
ouvrage4 = "4240-01D"
ouvrage5 = "4380-01D"
OUVRAGES = [ouvrage1,ouvrage2,ouvrage3,ouvrage4,ouvrage5]

data_surverse_init = filter(row -> row.NO_OUVRAGE ∈ OUVRAGES, data_surverse_init)
data_surverse = filter(row -> row.NO_OUVRAGE ∈ OUVRAGES, data_surverse)
ns = size(data_surverse,1)
print("")

## Chargement des précipitations

In [11]:
data_precipitation = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data_precipitation, Symbol("St-Hubert")=>:StHubert)
print("")

## Nettoyage des données sur les précipitations

#### Extraction des précipitations des mois de mai à octobre inclusivement

In [12]:
data_precipitation = filter(row -> month(row.date) > 4, data_precipitation) 
data_precipitation = filter(row -> month(row.date) < 11, data_precipitation) 
print("")

# Création des ensembles de test de validation

In [33]:
ratio = size(filter(row -> row.SURVERSE == 1, data_surverse),1) / size(data_surverse,1)

function séparation()
    b = Bernoulli(0.90)  # permet d'avoir un ensemble de validation de la même taille que l'ensemble de test   
    #b = Bernoulli(1)      # mettre 1 pour le test final (pour ne pas avoir d'ensemble de validation)
    ns = size(data_surverse,1)
    V = rand(b,ns)
    data_surverse[!,:b] = V

    surverse_train_init = filter(row -> row.b == 1, data_surverse) 
    surverse_valid_init = filter(row -> row.b == 0, data_surverse) 

    select!(data_surverse, [:NO_OUVRAGE, :DATE, :SURVERSE])
    select!(surverse_train_init, [:NO_OUVRAGE, :DATE, :SURVERSE])
    select!(surverse_valid_init, [:NO_OUVRAGE, :DATE, :SURVERSE])
    return surverse_train_init, surverse_valid_init
end

surverse_train_init, surverse_valid_init = séparation()
ratiobis = size(filter(row -> row.SURVERSE == 1, surverse_valid_init),1) / size(surverse_valid_init,1)

# cette boucle permet de s'assuer que l'ensemble de validation est représentatif de l'ensemble d'entrainement
while (ratiobis < ratio - 0.03) & (ratiobis > ratio + 0.05)
    surverse_train_init, surverse_valid_init = séparation()
    ratiobis = size(filter(row -> row.SURVERSE == 1, surverse_valid_init),1) / size(surverse_valid_init,1)
end


n_train = size(surverse_train_init,1)
n_valid = size(surverse_valid_init,1)
print(n_valid)
print('\n')
print(ns - (n_train + n_valid))

515
0

# Preprocessing

A chaque fois que l'on modifie un coefficient dans les cellules en dessuous, il suffit de relancer le code à partir d'ici ! (à part si on veut changer l'ensemble de test et de validation)

In [34]:
surverse_train = copy(surverse_train_init)
surverse_valid = copy(surverse_valid_init)
surverse_df_init = copy(data_surverse_init)
print("")

#### On remplace les valeurs de précipitation missing par une combinaison linéaire des données des autres stations

In [35]:
precipitation_df = copy(data_precipitation)

#McTavish, Bellevue, Assomption, Trudeau, StHubert
n_precipitation = size(precipitation_df,1)

COEFS=[[0,2,1,100,100],[2,0,1,100,1],[2,1,0,1,2],[100,100,1,0,2],[100,1,1,2,0]]  #on a déterminé ces 
#coefficients en se basant sur les distances sur la carte, on veut qu'une donnée manquante d'une station soit
#en priorité remplacée pas les données ce ses stations les plus proches.
COEFS=[[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]]             #permet d'effectuer une moyenne
#classique. Etonament l'accuracy est meilleure avec cette méthode.

for j=1:5
    coefs = COEFS[j]
    for i=1:n_precipitation
        if isequal(missing, precipitation_df[i,j+2])
            s = 0
            c = 0
            for k=1:5
                precipitation_df[i,k+2]
                if isequal(missing, precipitation_df[i,k+2])
                    s=s
                else
                    s += precipitation_df[i,k+2] * coefs[k] 
                    c += coefs[k] 
                end
            end
            if c == 0
                precipitation_df[i,j+2]  = 0
            else 
                precipitation_df[i,j+2] = floor(s/c)
            end
        end
    end
end

#### On ajoute des données de surverses positives qui sont les données de surverse que l'on a actuellement avec du bruit. Cela permet d'avoir des données de surverse en quantité plus nombreuse par rapport aux données de non-surverse et donc de rendre nottre classificateur plus efficace.

In [36]:
coeff = 10 #pour chaque donnée de surverse que l'on a on ajoute autant de donneés avec du bruit que le
#coefficient l'indique
#pour 10 on a environ autant de données avec une surverse que de données sans surverse

for i=1:n_train
    if surverse_train[i, :SURVERSE] == 1
        for k=1:coeff
            ouvrage = surverse_train[i, :NO_OUVRAGE]
            date = surverse_train[i, :DATE]
            year = Dates.year(date)
            month = Dates.month(date)
            day = Dates.day(date)
            new_date = Date(year - 10*k,month,day)
            ind = findfirst(precipitation_df[:,:date] .== date)
            while precipitation_df[ind,:date] .== date
                hour = precipitation_df[ind,:heure]
                McTavish = precipitation_df[ind,:McTavish]
                Bellevue = precipitation_df[ind,:Bellevue]
                Assomption = precipitation_df[ind,:Assomption]
                Trudeau = precipitation_df[ind,:Trudeau]
                StHubert = precipitation_df[ind,:StHubert]
                stations = [McTavish, Bellevue, Assomption, Trudeau, StHubert]
                for j=1:5
                    if stations[j] != 0
                        noise = Int64(floor(10 * (rand() - 0.5) + rand()))    #on utilise une loi uniforme pour
#le bruit qui semble mieux marcher qu'une loi normale
                        if stations[j] + noise >=0
                            stations[j] += noise
                        end
                    end
                end
                push!(precipitation_df,[new_date, hour, stations[1], stations[2], stations[3], stations[4], stations[5]])
                ind += 1
            end        
            push!(surverse_train,[ouvrage, new_date, 1]) 
            if size(filter(row -> row.DATE == new_date, surverse_df_init),1)==0
                push!(surverse_df_init,[ouvrage1, new_date, 1])
            end
        end
    end
end

In [37]:
print("taille des données de surverse:\n")
print(size(surverse_train,1))
print("\ntaille des données qui n'ont pas de surverses :\n")
print(size(filter(row -> row.SURVERSE == 0, surverse_train),1))
print("\ntaille des données qui ont surverses :\n")
print(size(filter(row -> row.SURVERSE == 1, surverse_train),1))

taille des données de surverse:
8514
taille des données qui n'ont pas de surverses :
4224
taille des données qui ont surverses :
4290

# Analyse exploratoire

Cette section consitue une analyse exploratoire superficielle permettant de voir s'il existe un lien entre les précipitations et les surverses.

Prenons arbitrairement l'ouvrage de débordement près du Bota-Bota (4350-01D). La station météorologique la plus proche est McTavish. Prenons deux variables explicatives simple :
- la somme journalière des précipitations
- le taux horaire maximum journalier de précipitations

#### Calcul de la quantité journalière de précipitations pour chacune des stations météorologiques

In [38]:
pcp_sum = by(precipitation_df, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)
first(pcp_sum ,5)
print("")

#### Extraction du taux horaire journalier maximum des précipitations pour chacune des stations météorologiques

In [39]:
pcp_max = by(precipitation_df, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)
first(pcp_max,5)
print("")

Au lieu de faire la somme sur toute la journée on fait la somme sur les 2 ou 3 ou il pleut le plus
Cependant ces variables n'augmentent pas l'accuracy de notre classificateur

In [40]:
n_precipitation = size(precipitation_df,1)
n_jour = size(pcp_max,1)

pcp_sum2h = copy(pcp_max)
pcp_sum2h[!, :McTavish] = zeros(Int64, n_jour)
pcp_sum2h[!, :Bellevue] = zeros(Int64, n_jour)
pcp_sum2h[!, :Assomption] = zeros(Int64, n_jour)
pcp_sum2h[!, :Trudeau] = zeros(Int64, n_jour)
pcp_sum2h[!, :StHubert] = zeros(Int64, n_jour)

for i=1:n_jour
    date = pcp_sum2h[i, :date]
    ind_init = findfirst(precipitation_df[:,:date] .== date)
    for j=1:5
        ind = ind_init
        max = 0
        jour1 = 0
        jour2 = 0
        while (ind < n_precipitation) & (precipitation_df[ind,:date] == date)
            jour1 = jour2
            jour2 = precipitation_df[ind, j+2]
            s = jour1 + jour2
            if s > max
                max = s
                pcp_sum2h[i, j+1] = max
            end
            ind += 1
        end
    end
end
print("")

In [41]:
n_precipitation = size(precipitation_df,1)
n_jour = size(pcp_max,1)

pcp_sum3h = copy(pcp_max)
pcp_sum3h[!, :McTavish] = zeros(Int64, n_jour)
pcp_sum3h[!, :Bellevue] = zeros(Int64, n_jour)
pcp_sum3h[!, :Assomption] = zeros(Int64, n_jour)
pcp_sum3h[!, :Trudeau] = zeros(Int64, n_jour)
pcp_sum3h[!, :StHubert] = zeros(Int64, n_jour)

for i=1:n_jour
    date = pcp_sum3h[i, :date]
    ind_init = findfirst(precipitation_df[:,:date] .== date)
    for j=1:5
        ind = ind_init
        max = 0
        jour1 = 0
        jour2 = 0
        jour3 = 0
        while (ind < n_precipitation) & (precipitation_df[ind,:date] == date)
            jour1 = jour2
            jour2 = jour3
            jour3 = precipitation_df[ind, j+2]
            s = jour1 + jour2 + jour3
            if s > max
                max = s
                pcp_sum3h[i, j+1] = max
            end
            ind += 1
        end
    end
end
print("")

## Entrainement

Nous avons choisi d'utiliser une régression logistique car les données à derterminer sont binaires et donc ce modèle s'applique bien. 
Après avoir testé différentes combinaisons de variables, celles qui fonctionnent le mieux sont les 5 variables de somme et les 5 variables de maximum. Nous mettons les variables des 5 stations même si certaines sont plus proches que d'autres car notre fonction de régression se chargera d'attribuer des coefficients de régression plus ou moins important à chaque station. 

In [42]:
df0 = filter(row -> row.NO_OUVRAGE == ouvrage1, surverse_df_init)
df1 = filter(row -> row.NO_OUVRAGE == ouvrage1, surverse_train)
df2 = filter(row -> row.NO_OUVRAGE == ouvrage2, surverse_train)
df3 = filter(row -> row.NO_OUVRAGE == ouvrage3, surverse_train)
df4 = filter(row -> row.NO_OUVRAGE == ouvrage4, surverse_train)
df5 = filter(row -> row.NO_OUVRAGE == ouvrage5, surverse_train)

n = size(df0,1) #max des size des dfs

x1 = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x3 = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x5 = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x7 = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x9 = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x2 = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x4 = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x6 = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x8 = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x10 = Array{Union{Missing, Int64}}(missing, n) # variable pour les 3 hueres d'affilées
x11 = Array{Union{Missing, Int64}}(missing, n) # variable pour les 3 hueres d'affilées
x12 = Array{Union{Missing, Int64}}(missing, n) # variable pour les 3 hueres d'affilées
x13 = Array{Union{Missing, Int64}}(missing, n) # variable pour les 3 hueres d'affilées
x14 = Array{Union{Missing, Int64}}(missing, n) # variable pour les 3 hueres d'affilées
x15 = Array{Union{Missing, Int64}}(missing, n) # variable pour les 3 hueres d'affilées

for i=1:size(df0,1)
    
    ind1 = findfirst(pcp_sum[:,:date] .== df0[i,:DATE])
    ind2 = findfirst(pcp_max[:,:date] .== df0[i,:DATE])
    
    #McTavish
    x1[i] = pcp_sum[ind1,:McTavish]
    x2[i] = pcp_max[ind2,:McTavish]
    #x11[i] = pcp_3h[ind2,:McTavish]
    
    #Bellevue
    x3[i] = pcp_sum[ind1,:Bellevue]
    x4[i] = pcp_max[ind2,:Bellevue]
    #x12[i] = pcp_3h[ind2,:Bellevue]
    
    #Assomption
    x5[i] = pcp_sum[ind1,:Assomption]
    x6[i] = pcp_max[ind2,:Assomption]
    #x13[i] = pcp_3h[ind2,:Assomption]
    
    #Trudeau
    x7[i] = pcp_sum[ind1,:Trudeau]
    x8[i] = pcp_max[ind2,:Trudeau]
    #x14[i] = pcp_3h[ind2,:Trudeau]
    
    #StHubert
    x9[i] = pcp_sum[ind1,:StHubert]
    x10[i] = pcp_max[ind2,:StHubert]
    #x15[i] = pcp_3h[ind2,:StHubert]
    
end

Nous avons essayé d'introduire une variable pour la saison comme suit, cependant cela n'améliore pas la prédiction.

In [43]:
x21 = zeros(Int64, n)  #printemps
x22 = zeros(Int64, n)  #automne

for i=1:size(df0,1)
    if month(df0[i,:DATE]) < 7 #on est au printemps
        x11[i] = 1
    elseif month(df0[i,:DATE]) > 9 #on est en automne
        x12[i] = 1
    end
end

In [44]:
DF = [df1,df2,df3,df4,df5]
for df in DF
    x1c=copy(x1)
    x2c=copy(x2)
    x3c=copy(x3)
    x4c=copy(x4)
    x5c=copy(x5)
    x6c=copy(x6)
    x7c=copy(x7)
    x8c=copy(x8)
    x9c=copy(x9)
    x10c=copy(x10)
    x11c=copy(x11)
    x12c=copy(x12)
    x13c=copy(x13)
    x14c=copy(x14)
    x15c=copy(x15)
    Xc = [x1c,x2c,x3c,x4c,x5c,x6c,x7c,x8c,x9c,x10c,x11c,x12c,x13c,x14c,x15c]
    for j=1:size(df0,1)
        i = size(df0,1) + 1 - j
        date = df0[i,:DATE]
        if size(filter(row -> row.DATE == date, df),1)==0
            for k=1:size(Xc,1)
                deleteat!(Xc[k], i)
            end
        end
    end
    df[!,:x1] = x1c
    df[!,:x2] = x2c
    df[!,:x3] = x3c
    df[!,:x4] = x4c
    df[!,:x5] = x5c
    df[!,:x6] = x6c
    df[!,:x7] = x7c
    df[!,:x8] = x8c
    df[!,:x9] = x9c
    df[!,:x10] = x10c
    df[!,:x11] = x11c
    df[!,:x12] = x12c
    df[!,:x13] = x13c
    df[!,:x14] = x14c
    df[!,:x15] = x15c
end

In [45]:
M1 = glm(@formula(SURVERSE ~ x1+x2+x3+x4+x5+x6+x7+x8+x9+x10), df1,  Bernoulli(), LogitLink())
M2 = glm(@formula(SURVERSE ~ x1+x2+x3+x4+x5+x6+x7+x8+x9+x10), df2,  Bernoulli(), LogitLink())
M3 = glm(@formula(SURVERSE ~ x1+x2+x3+x4+x5+x6+x7+x8+x9+x10), df3,  Bernoulli(), LogitLink())
M4 = glm(@formula(SURVERSE ~ x1+x2+x3+x4+x5+x6+x7+x8+x9+x10), df4,  Bernoulli(), LogitLink())
M5 = glm(@formula(SURVERSE ~ x1+x2+x3+x4+x5+x6+x7+x8+x9+x10), df5,  Bernoulli(), LogitLink())
print("")

## Validation

Ne pas exécuter les cellules qui suivent si l'on a pas créé d'ensemble de validation.

In [46]:
df1v = filter(row -> row.NO_OUVRAGE == ouvrage1, surverse_valid)
df2v = filter(row -> row.NO_OUVRAGE == ouvrage2, surverse_valid)
df3v = filter(row -> row.NO_OUVRAGE == ouvrage3, surverse_valid)
df4v = filter(row -> row.NO_OUVRAGE == ouvrage4, surverse_valid)
df5v = filter(row -> row.NO_OUVRAGE == ouvrage5, surverse_valid)

nv = size(df0,1)

x1v = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x3v = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x5v = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x7v = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x9v = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x2v = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x4v = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x6v = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x8v = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x10v = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier

for i=1:nv
    
    ind1 = findfirst(pcp_sum[:,:date] .== df0[i,:DATE])
    ind2 = findfirst(pcp_max[:,:date] .== df0[i,:DATE])
    
    #McTavish
    x1v[i] = pcp_sum[ind1,:McTavish]
    x2v[i] = pcp_max[ind2,:McTavish]
    
    #Bellevue
    x3v[i] = pcp_sum[ind1,:Bellevue]
    x4v[i] = pcp_max[ind2,:Bellevue]
    
    #Assomption
    x5v[i] = pcp_sum[ind1,:Assomption]
    x6v[i] = pcp_max[ind2,:Assomption]
    
    #Trudeau
    x7v[i] = pcp_sum[ind1,:Trudeau]
    x8v[i] = pcp_max[ind2,:Trudeau]
    
    #StHubert
    x9v[i] = pcp_sum[ind1,:StHubert]
    x10v[i] = pcp_max[ind2,:StHubert]
    
end

In [47]:
df01v = copy(df0[!,:DATE])
df02v = copy(df0[!,:DATE])
df03v = copy(df0[!,:DATE])
df04v = copy(df0[!,:DATE])
df05v = copy(df0[!,:DATE])
DF0 = [df01v,df02v,df03v,df04v,df05v]

DF = [df1v,df2v,df3v,df4v,df5v]
for k=1:5
    df = DF[k]
    x1c=copy(x1v)
    x2c=copy(x2v)
    x3c=copy(x3v)
    x4c=copy(x4v)
    x5c=copy(x5v)
    x6c=copy(x6v)
    x7c=copy(x7v)
    x8c=copy(x8v)
    x9c=copy(x9v)
    x10c=copy(x10v)
    x11c=copy(x11)
    x12c=copy(x12)
    Xc = [x1c,x2c,x3c,x4c,x5c,x6c,x7c,x8c,x9c,x10c,x11c,x12c]
    for j=1:size(df0,1)
        i = size(df0,1) + 1 - j
        date = df0[i,:DATE]
        if size(filter(row -> row.DATE == date, df),1)==0
            for xic in Xc
                deleteat!(xic, i)
            end
            deleteat!(DF0[k],i)
        end
    end
    df[!,:x0] = ones(Int64,length(x1c))
    df[!,:x1] = x1c
    df[!,:x2] = x2c
    df[!,:x3] = x3c
    df[!,:x4] = x4c
    df[!,:x5] = x5c
    df[!,:x6] = x6c
    df[!,:x7] = x7c
    df[!,:x8] = x8c
    df[!,:x9] = x9c
    df[!,:x10] = x10c
    df[!,:x11] = Xc[11]
    df[!,:x12] = Xc[12]
end

In [48]:
select!(df1v, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta1v = predict(M1,df1v) 
select!(df2v, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta2v = predict(M2,df2v) 
select!(df3v, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta3v = predict(M3,df3v) 
select!(df4v, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta4v = predict(M4,df4v) 
select!(df5v, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta5v = predict(M5,df5v) 
print("")

In [49]:
Y1v = zeros(Int64,size(df1v,1))
Y2v = zeros(Int64,size(df2v,1))
Y3v = zeros(Int64,size(df3v,1))
Y4v = zeros(Int64,size(df4v,1))
Y5v = zeros(Int64,size(df5v,1))

Y1v[teta1v.>.5] .= 1
Y2v[teta2v.>.5] .= 1
Y3v[teta3v.>.5] .= 1
Y4v[teta4v.>.5] .= 1
Y5v[teta5v.>.5] .= 1
print("")

Permet de calculer le Fscore sur l'ensemble de validation.

In [50]:
TP = 0
FP = 0
FN = 0
TN = 0

for i=1:n_valid
    ouvrage = surverse_valid[i,:NO_OUVRAGE]
    date = surverse_valid[i,:DATE]
    if ouvrage == ouvrage1
        ind = findfirst(df01v[:] .== date)
        prediction = Y1v[ind]
    end
    if ouvrage == ouvrage2
        ind = findfirst(df02v[:] .== date)
        prediction = Y2v[ind]
    end
    if ouvrage == ouvrage3
        ind = findfirst(df03v[:] .== date)
        prediction = Y3v[ind]
    end
    if ouvrage == ouvrage4
        ind = findfirst(df04v[:] .== date)
        prediction = Y4v[ind]
    end
    if ouvrage == ouvrage5
        ind = findfirst(df05v[:] .== date)
        prediction = Y5v[ind]
    end
    
    if surverse_valid[i,:SURVERSE] == 1
        if prediction == 1
            TP += 1
        else
            FN += 1
        end
    else
        if prediction == 1
            FP += 1
        else
            TN += 1
        end
    end
    
end

precision = TP / (TP + FP)
recall = TP / (TP + FN)

score = 2 * precision * recall / (precision + recall)
print("Le Fscore obtenu est : ", score)

Le Fscore obtenu est : 0.7777777777777777

#### Traçage des distribution de la somme des précipitations en fonction des surverses ou non

On remarque que les deux distributions sont très différentes. Ceci suggère que la somme des précipitations à la station McTavish a un effet sur les surverses au Bota-Bota.

In [51]:
#plot(df1, x=:SURVERSE, y=:x1, Geom.boxplot)

#### Traçage des distribution de la somme des précipitations en fonction des surverses ou non

On remarque que les deux distributions sont très différentes. Ceci suggère que le maximum journalier des précipitations à la station McTavish a un effet sur les surverses au Bota-Bota.

In [17]:
#plot(df, x=:SURVERSE, y=:MAX, Geom.boxplot)

# Création du fichier de prédictions pour soumettre sur Kaggle

Dans ce cas-ci, nous prédirons une surverse avec une probabilité de 1/2 sans considérer aucune variable explicative.

In [52]:
# Chargement du fichier de test
test = CSV.read("data/test.csv")

# Pour chacune des lignes du fichier test, comportant un ouvrage et une date, une prédiction est requise.
# Dans ce cas-ci, utilisons une prédiction les plus naîve. 
# On prédit avec une chance sur deux qu'il y ait surverse, sans utiliser de variables explicatives
n0 = size(test,1)

283

In [54]:
df1_ = filter(row -> row.NO_OUVRAGE == ouvrage1, test)
df2_ = filter(row -> row.NO_OUVRAGE == ouvrage2, test)
df3_ = filter(row -> row.NO_OUVRAGE == ouvrage3, test)
df4_ = filter(row -> row.NO_OUVRAGE == ouvrage4, test)
df5_ = filter(row -> row.NO_OUVRAGE == ouvrage5, test)

df0_ = copy(df1_[!,:DATE])
for df_ in [df2_, df3_, df4_, df5_]
    for i=1:size(df_,1)
        if (df_[i,:DATE] in df0_)==false
            push!(df0_, df_[i,:DATE])
        end
    end
end

n_ = length(df0_)

x1_ = Array{Union{Missing, Int64}}(missing, n_) # variable pour la somme journalière
x3_ = Array{Union{Missing, Int64}}(missing, n_) # variable pour la somme journalière
x5_ = Array{Union{Missing, Int64}}(missing, n_) # variable pour la somme journalière
x7_ = Array{Union{Missing, Int64}}(missing, n_) # variable pour la somme journalière
x9_ = Array{Union{Missing, Int64}}(missing, n_) # variable pour la somme journalière
x2_ = Array{Union{Missing, Int64}}(missing, n_) # variable pour le max journalier
x4_ = Array{Union{Missing, Int64}}(missing, n_) # variable pour le max journalier
x6_ = Array{Union{Missing, Int64}}(missing, n_) # variable pour le max journalier
x8_ = Array{Union{Missing, Int64}}(missing, n_) # variable pour le max journalier
x10_ = Array{Union{Missing, Int64}}(missing, n_) # variable pour le max journalier

for i=1:n_
    
    ind1 = findfirst(pcp_sum[:,:date] .== df0_[i])
    ind2 = findfirst(pcp_max[:,:date] .== df0_[i])
    
    #McTavish
    x1_[i] = pcp_sum[ind1,:McTavish]
    x2_[i] = pcp_max[ind2,:McTavish]
    
    #Bellevue
    x3_[i] = pcp_sum[ind1,:Bellevue]
    x4_[i] = pcp_max[ind2,:Bellevue]
    
    #Assomption
    x5_[i] = pcp_sum[ind1,:Assomption]
    x6_[i] = pcp_max[ind2,:Assomption]
    
    #Trudeau
    x7_[i] = pcp_sum[ind1,:Trudeau]
    x8_[i] = pcp_max[ind2,:Trudeau]
    
    #StHubert
    x9_[i] = pcp_sum[ind1,:StHubert]
    x10_[i] = pcp_max[ind2,:StHubert]
    
end

In [55]:
#variables pour la saison

x11_ = zeros(Int64, n_)  #printemps
x12_ = zeros(Int64, n_)  #automne

for i=1:n_
    if month(df0_[i]) < 7 #on est au printemps
        x11_[i] = 1
    elseif month(df0_[i]) > 9 #on est en automne
        x12_[i] = 1
    end
end

In [56]:
df01 = copy(df0_)
df02 = copy(df0_)
df03 = copy(df0_)
df04 = copy(df0_)
df05 = copy(df0_)
DF0 = [df01,df02,df03,df04,df05]

DF = [df1_,df2_,df3_,df4_,df5_]
for k=1:5
    df_ = DF[k]
    x1_c=copy(x1_)
    x2_c=copy(x2_)
    x3_c=copy(x3_)
    x4_c=copy(x4_)
    x5_c=copy(x5_)
    x6_c=copy(x6_)
    x7_c=copy(x7_)
    x8_c=copy(x8_)
    x9_c=copy(x9_)
    x10_c=copy(x10_)
    x11_c=copy(x11_)
    x12_c=copy(x12_)
    X_c = [x1_c,x2_c,x3_c,x4_c,x5_c,x6_c,x7_c,x8_c,x9_c,x10_c,x11_c,x12_c]
    for j=1:length(df0_)
        i = length(df0_) + 1 - j
        date = df0_[i]
        if size(filter(row -> row.DATE == date, df_),1)==0
            for xi_c in X_c
                deleteat!(xi_c, i)
            end
            deleteat!(DF0[k],i)
        end
    end
    df_[!,:x0] = ones(Int64,length(x1_c))
    df_[!,:x1] = x1_c
    df_[!,:x2] = x2_c
    df_[!,:x3] = x3_c
    df_[!,:x4] = x4_c
    df_[!,:x5] = x5_c
    df_[!,:x6] = x6_c
    df_[!,:x7] = x7_c
    df_[!,:x8] = x8_c
    df_[!,:x9] = x9_c
    df_[!,:x10] = x10_c
    df_[!,:x11] = X_c[11]
    df_[!,:x12] = X_c[12]
end

In [57]:
select!(df1_, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta1 = predict(M1,df1_) 
select!(df2_, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta2 = predict(M2,df2_) 
select!(df3_, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta3 = predict(M3,df3_) 
select!(df4_, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta4 = predict(M4,df4_) 
select!(df5_, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta5 = predict(M5,df5_) 
print("")

In [58]:
Y1 = zeros(Int64,size(df1_,1))
Y2 = zeros(Int64,size(df2_,1))
Y3 = zeros(Int64,size(df3_,1))
Y4 = zeros(Int64,size(df4_,1))
Y5 = zeros(Int64,size(df5_,1))

Y1[teta1.>.5] .= 1
Y2[teta2.>.5] .= 1
Y3[teta3.>.5] .= 1
Y4[teta4.>.5] .= 1
Y5[teta5.>.5] .= 1
print("")

In [59]:
#Création de surverse test :
surverse = zeros(Int64, n0)
for i=1:n0
    ouvrage = test[i,:NO_OUVRAGE]
    date = test[i,:DATE]
    if ouvrage == ouvrage1
        ind = findfirst(df01[:] .== date)
        surverse[i] = Y1[ind]
    end
    if ouvrage == ouvrage2
        ind = findfirst(df02[:] .== date)
        surverse[i] = Y2[ind]
    end
    if ouvrage == ouvrage3
        ind = findfirst(df03[:] .== date)
        surverse[i] = Y3[ind]
    end
    if ouvrage == ouvrage4
        ind = findfirst(df04[:] .== date)
        surverse[i] = Y4[ind]
    end
    if ouvrage == ouvrage5
        ind = findfirst(df05[:] .== date)
        surverse[i] = Y5[ind]
    end
end

In [60]:
#print(test)

In [61]:
#surverse = rand(n0) .> .5

# Création du fichier sampleSubmission.csv pour soumettre sur Kaggle
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=surverse)
CSV.write("sampleSubmission.csv",sampleSubmission)

# Vous pouvez par la suite déposer le fichier sampleSubmission.csv sur Kaggle.


"sampleSubmission.csv"